Este _script_ lee un PDF definido y extrae todas aquellas vacantes que están en tablas de 15 columas.  
Los cursos, tiempos máximos y mínimos, fecha de cobertura y otros campos que no están definidos para todas las vacantes, el _script_ genera una lista de posibles valores, ya que la estructura del PDF del BOD no crea filas en las tablas, sólo una columna.

In [131]:
import pandas as pd
import numpy as np
import tabula
import math
import os

In [2]:
file_pdf = 'D:\jaume\Datasets\BOD\BOD_20210325_58_vacantes_CM.pdf'
read_pdf = tabula.read_pdf(file_pdf, pages = 'all', silent = True)

In [3]:
sel_tables = []                         #creará una lista (sel_tables) con los índices a mantener basado en el número de columnas de la tabla (15)
for i in range(len(read_pdf)):
    if read_pdf[i].shape[1] == 15:         #seleccionamos sólo los df que contienen 15 columnas
        if 'LD' in read_pdf[i].iloc[1,4]:  #eliminamos los df que en la 5ª columna contienen la cadena LD (son vacantes que podremos analizar luego, poco comunes y con columnas cambiadas)
            continue
        else:
            sel_tables.append(i)
    else:
        continue

In [4]:
df_list = [read_pdf[index] for index in sel_tables]    #conservará los elementos de read_tables seleccionados en sel_tables
len(df_list)

30

In [141]:
df_clean = []
errors = []
for i in range(len(df_list)):
    df = df_list[i]
    n_vac = df.iloc[1,0]\
              .split('\r')
    uco = df.iloc[1, 1]\
            .split('\r')
    uco = uco[0::2]
    ciu = df.iloc[1, 2]\
            .split('\r')
    pt = df.iloc[1, 3]\
           .split('\r')
    clas = df.iloc[1, 4]\
             .split('\r')
    ta = df.iloc[1, 5]\
           .split('\r')
    empleo = df.iloc[1, 6]\
               .split('\r')
    efun = str(df.iloc[1, 7])\
             .split('\r')

    csce = df.iloc[1, 12]\
             .split('\r')
    cod_cm = df.iloc[1, 13]\
               .split('\r')
    obs = df.iloc[1, 14]\
            .split('\r')

    if str(df.iloc[1,8]) == np.nan:          #si no hay cursos, todos tendránun np.nan
        cursos = [np.nan] * len(n_vac)
    else:
        cursos = str(df.iloc[1, 8])\
                       .split('\r')      #si los hay, separamos las opciones
    
    if str(df.iloc[1,9]) == np.nan:
        t_max = [np.nan] * len(n_vac)
    else:
        t_max = str(df.iloc[1, 9])\
                  .split('\r')       #
    
    if str(df.iloc[1,10]) == '':
        t_min = [np.nan] * len(n_vac)
    else:
        t_min = str(df.iloc[1, 10])\
                  .split('\r')       # 

    if str(df.iloc[1,11]) == '':
        fecha_cob = [np.nan] * len(n_vac)
    else:
        fecha_cob = str(df.iloc[1, 11])\
                          .split('\r')       # 

    #La longitud de las listas de cursos, t_max, t_min no cuadran con el resto
    #(porque no todas las vacantesd los tienen), estos bloques se presentartán 
    #como listas con los posibles valores para toda la tabla
    if len(cursos) == len(n_vac):
        pass
    else:
        errors.append('DF-' + str(i) + ' CURSOS no definido en todas las vacantes. Ver opciones.')
        unique_cursos = set(cursos)
        cursos_value = list(unique_cursos)
        cursos_value.append(np.nan)
        cursos = [cursos_value] * len(n_vac)
    
    if len(t_max) == len(n_vac):
        pass
    else:
        errors.append('DF-' + str(i) + ' T_MAX no definido en todas las vacantes. Ver opciones.')
        unique_tmax = set(t_max)
        tmax_value = list(unique_tmax)
        tmax_value.append(np.nan)
        t_max = [tmax_value] * len(n_vac)
        
    if len(t_min) == len(n_vac):
        pass
    else:
        errors.append('DF-' + str(i) + ' T_MIN no definido en todas las vacantes. Ver opciones.')
        unique_tmin = set(t_min)
        tmin_value = list(unique_tmin)
        tmin_value.append(np.nan)
        t_min = [tmin_value] * len(n_vac)

    if len(efun) == len(n_vac):
        pass
    else:
        errors.append('DF-' + str(i) + ' EFUN no está definida en todas las vacantes')
        unique_efun = set(efun)    #devuelve valores únicos de EFUN en un set (de valores únicos)
        efun_value = list(unique_efun)
        efun_value.append(np.nan)            #lista de posibles EFUN
        efun = [efun_value] * len(n_vac)    #lista de longiutud igual a nº de vacantes con las posibles EFUN
        
    if len(fecha_cob) == len(n_vac):
        pass
    else:
        errors.append('DF-' + str(i) + ' FECHA DE COBERTURA no está definida en todas las vacantes')
        unique_fechaCob = set(fecha_cob)    #devuelve valores únicos de EFUN en un set (de valores únicos)
        fechaCob_value = list(unique_fechaCob)
        fechaCob_value.append(np.nan)            #lista de posibles EFUN
        fecha_cob = [fechaCob_value] * len(n_vac)    #lista de longiutud igual a nº de vacantes con las posibles EFUN
    
    data_dict = {'n_vac': n_vac,
                 'uco': uco,
                 'ciu': ciu,
                 'pt': pt,
                 'clas': clas,
                 'ta': ta,
                 'empleo': empleo,
                 'efun': efun,
                 'cursos': cursos,
                 't_max': t_max,
                 't_min': t_min,
                 'fecha_cob': fecha_cob,
                 'csce': csce,
                 'cod_cm': cod_cm,
                 'obs': obs
                }

    try:
        df_clean.append(pd.DataFrame(data_dict))
    except:
        errors.append('DF-' + str(i) + ' threw an error in DataFrame()')

In [142]:
str(len(df_clean)) + ' clean df,s in the list, ready to pack'

'30 clean df,s in the list, ready to pack'

# Pack the df,s!

In [154]:
df_full = pd.concat(df_clean).reset_index(drop = True)      #la solución!!!

# Pickle-it!

In [155]:
pkl_dir = 'D:/jaume/Jupyter Notebooks/Vacantes Scraper/ScrapedData/'
pkl_name_parts = file_pdf.split('_')
pkl_name_cm = pkl_name_parts[1] + '_' + pkl_name_parts[2] + '_CM.pkl'

df_full.to_pickle(pkl_dir + pkl_name_cm)

In [156]:
#errors

In [157]:
df_full

,n_vac,uco,ciu,pt,clas,ta,empleo,efun,cursos,t_max,t_min,fecha_cob,csce,cod_cm,obs
0,03800,BATALLON DE HELICOPTEROS DE EMERGENCIAS II (BE...,50075516,5SA3E/003,CM,C,STTE a BG,"[I.LIG, ARTC, nan]","HC:I, HR:I","[10, nan]","[3, nan]","[nan, nan]","304,34",GM18,"546,656,959"
1,03801,BATALLON DE CG. DE LAS FAMET,50045588,5SA3E/002,CM,C,STTE a BG,"[I.LIG, ARTC, nan]",HP:E,"[10, nan]","[3, nan]","[nan, nan]","374,87",GM18,"546,656,959"
2,03802,BATALLON DE HELICOPTEROS DE EMERGENCIAS II (BE...,50075516,5SA3E/004,CM,C,STTE a BG,"[I.LIG, ARTC, nan]",HP:E,"[10, nan]","[3, nan]","[nan, nan]","374,87",GM18,"546,656,959"
3,03803,BATALLON DE HELICOPTEROS DE MANIOBRA VI,50030082,5SA3E/003,CM,C,STTE a BG,"[I.LIG, ARTC, nan]",HP:E,"[10, nan]","[3, nan]","[nan, nan]","374,87",GM18,"546,656,959"
4,03804,RGTO. DE INF. «AMERICA» N.o 66 DE CAZADORES DE...,50038946,5SA19/002,CM,C,STTE,"[I.LIG, ARTC, nan]",TM:E,"[10, nan]","[3, nan]","[nan, nan]","374,87",GM18,959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,03034,DIRECCION DE ADQUISICIONES,57655773,5JA03/006,CM,C,TCOL,"[ART., ADMON, CAB., TRA., nan]","[nan, nan]","[4, 10, nan]","[2.0, nan]","[nan, nan]","450,47",GM05,"656,960"
612,03035,"DIRECCION DE INVESTIGACIÓN, DOCTRINA, ORGANICA...",55230000,5JA33/002,CM,C,TCOL a CTE,"[ART., ADMON, CAB., TRA., nan]","[nan, nan]","[4, 10, nan]","[2.0, nan]","[nan, nan]","374,87",GM05,"656,960"
613,03036,ACADEMIA DE CABALLERIA,55107701,5JA42/001,CM,C,CTE,"[ART., ADMON, CAB., TRA., nan]","[nan, nan]","[4, 10, nan]","[2.0, nan]","[nan, nan]","338,86",GM13,960
614,03037,"DIRECCION DE ENSEÑANZA, INS. ADI. EVAL. (GRANADA)",55100004,5JA39/002,CM,C,CTE,"[ART., ADMON, CAB., TRA., nan]","[nan, nan]","[4, 10, nan]","[2.0, nan]","[nan, nan]","374,87",GM13,"511,656,960"


In [149]:
math.isnan(df_list[0].iloc[1,11])

True